In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
"""
NOTE: combining 'ORI' and 'Incident Number' is not enough for a unique identifier per event. 
Waiting to hear clarification form Ben, but could create a key using date also. That should
be enough to join matching entries from one table to another correctly.
NOTE: Reply: heard back from Ben, got a new csv from a specific PD that only has 198 records, of which only 170 have unique INs 
"""

In [2]:
# Deliverables 1 & 2 Need to join:
# Data/2020 Domestic Violence - Administrative Data.csv
# Data/2020 Domestic Violence - Other Data.csv
# Data/2020 Q2 Domestic Violence - Location Type.csv
admin_df = pd.read_csv('../Data/2020 Domestic Violence - Administrative Data.csv', encoding='ISO-8859-1',skiprows=7)
other_df = pd.read_csv('../Data/2020 Domestic Violence - Other Data.csv', encoding='ISO-8859-1',skiprows=7)
location_df = pd.read_csv('../Data/2020 Q1 Domestic Violence - Location Type.csv', encoding='ISO-8859-1',skiprows=6)
#nadmin_df = pd.read_csv('../Data/new2020 Domestic Violence - Administrative for Athens PD 08192022.csv', encoding='ISO-8859-1',skiprows=6)


In [9]:
#len(admin_df), len(admin_df.drop_duplicates()), len(set(admin_df['Incident Number']))

(69299, 60054, 58893)

In [3]:
base_df = admin_df.drop_duplicates()
# MM/DD/YYYY -> need Day and Month specifically
base_df[['Month', 'Day','Year']] = base_df['Incident Date'].str.split('/', expand=True)
base_df[['Hour','End']] = base_df['Hour of Day'].str.split('-', expand=True)
defects = {
    '11:0 0am':'11:00am',
    '12:00n':'12:00pm',
    }
base_df.replace(to_replace=defects, inplace=True)
base_df.reset_index(drop=True, inplace=True)
base_df = base_df[base_df.Hour != 'Unknown']


/var/folders/pt/y3bb3b4j7yz3zzczhxc5l9p0vbrddc/T/ipykernel_41365/263674152.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_df[['Month', 'Day','Year']] = base_df['Incident Date'].str.split('/', expand=True)
/var/folders/pt/y3bb3b4j7yz3zzczhxc5l9p0vbrddc/T/ipykernel_41365/263674152.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_df[['Month', 'Day','Year']] = base_df['Incident Date'].str.split('/', expand=True)
/var/folders/pt/y3bb3b4j7yz3zzczhxc5l9p0vbrddc/T/ipykernel_41365/263674152.py:3: S

In [4]:
# # '4:00pm'
# x = base_df.iloc[13]['Hour']
# print(len(x))
# x1,p = x[:4],x[4:]
# x = ' '.join([x1,p])
# in_time = datetime.strptime(x, "%I:%M %p")
# out_time = datetime.strftime(in_time, "%H:%M")
# print(out_time)

def change_time_format(x):
    if len(x) == 6:
        x1,p = x[:4],x[4:]    
    else:
        x1,p = x[:5],x[5:]
    t = ' '.join([x1,p])
    in_time = datetime.strptime(t, "%I:%M %p")
    return datetime.strftime(in_time, "%H:%M")



In [ ]:
# # checking for errors
# new_hour = []
# i = 0
# for t in base_df.Hour:
#     try:
#         new_hour.append(change_time_format(t))
#     except ValueError:
#         print(i)
#         break
#     i += 1
# len(new_hour)


In [6]:
base_df.Hour = base_df.Hour.apply(change_time_format)
base_df.Day = base_df.Day.apply(int)
base_df.Month = base_df.Month.apply(int)

In [7]:
ts_df = base_df[['ORI','Incident Number','Month','Day','Year','Hour','Incident Clearance']]
ts_df.head()


,ORI,Incident Number,Month,Day,Year,Hour,Incident Clearance
0,TN0410000,20010101-133,1,1,2020,13:00,Not Cleared
1,TN0150100,2.02001E+11,1,2,2020,01:00,Cleared by Arrest
2,TN0150100,2.02001E+11,1,2,2020,11:00,Cleared by Arrest
3,TN0410000,200103-2142,1,3,2020,21:00,Cleared by Arrest
4,TN0500100,20010166,1,2,2020,11:00,Cleared by Arrest


In [8]:
ts_df.dtypes

ORI                   object
Incident Number       object
Month                  int64
Day                    int64
Year                  object
Hour                  object
Incident Clearance    object
dtype: object